# ZooKeeper

- [Home](https://zookeeper.apache.org/) - [doc](http://zookeeper.apache.org/doc/current/index.html)
  - ZooKeeper Programmer's Guide
  - ZooKeeper Internals
  - ZooKeeper-cli: the ZooKeeper command line interface
- [Builtin ACL Schemes](https://zookeeper.apache.org/doc/r3.7.2/zookeeperProgrammers.html#sc_BuiltinACLSchemes)

Code: `rtfsc\zookeeper`

actions:
- with Curator: workbench\DataEngineering\codes\data-engineering-java\database\zookeeper

More:
* [ZooKeeper.md](./ZooKeeper.md)

# Zab: the ZooKeeper Atomic Broadcast algorithm

papers:
* (Zab A) Junqueira, Flavio Paiva / Reed, Benjamin C. / Serafini, Marco. **Zab: High-performance broadcast for primary-backup systems**. 2011.
* (Zab B) Medeiros, André. **ZooKeeper’s atomic broadcast protocol: Theory and practice**. 2012.

假设:
* crash-recovery system model: 崩溃恢复

节点角色:
* leader
  * 发起和维护与follower和observer的心跳
  * 负责处理写操作, 采用类2PC协议广播给follower/observer, 超过半数follower写入成功, 提交操作.
* follower
  * 响应leader的心跳
  * 将写请求转发给leader处理
  * 可以直接处理读请求
* observer
  * 不参与广播中写入成功计数
  * 将写请求转发给leader处理
  * 可以直接处理读请求

节点状态:
* following
* leading
* election: 执行leader选举算法

数据结构:
* epoch: 整数, 表示在一段时间内leader存在 - leader周期
* transaction: 事务, ensemble中广播的状态变更, 由(v,z)对表示, v表示新状态, z为zxid标识符
  * 由leader/primary发出提案proposed
  * 由调用投递方法的进程投递/提交(dilivered/committed)
* 事务编号zxid: 由(e,c)对表示, e是生成该事务的primary的的epoch number, c表示事务计数器 
  * 每次产生一个新leader时, 从它的日志中取出最大的zxid, 将zxid.e+1作为新的epoch, c复位.
  * 排序: (e,c) < (e', c'): 如果e < e' 或者 e = e'且c < c'
* 节点的持久状态
  * history: 接受的事务提案的日志
  * accpetedEpoch: 最近接受的NEWEPOCH消息的epoch number
  * currentEpoch: 最近接受的NEWLEADER消息的epoch number
  * lastZxid: history中最近的提案的zxid


Zab协议的阶段: - (Zab B)
* 阶段0: Leader election
  * 执行leader选举算法, 投票
* 阶段1: Discovery
  * 阶段开始时, 查看投票, 决定自己是follower还是leader
* 阶段2: Synchronization
* 阶段3: Broadcast
  * 最多一个leader
  * 广播: 类似于2PC协议

说明:
* leader和follower依次执行Discovery, Synchronization, Broadcast这三个阶段.
* 阶段1和阶段2将ensemble恢复到一致的状态, 特别是从崩溃中恢复.
* 在阶段1,2,3中, 节点在出现失败或超时时, 可以重新进入阶段0执行Leader选举.
* Java实现
  * 选举阶段使用FastLeaderElection, 会选举出拥有最新提交事务提案的节点作为leader, 省去了发现被大多数接受的最新事务提案的步骤.
  * 将发现阶段和同步阶段合并成恢复阶段.
  * 只有三个阶段: Fast Leader Election, Recovery, Boradcast.

客户端:
* 提交操作到连接的节点, 如果是状态变更, Zab会执行广播.
  * 如果节点是follower: 转发到leader节点;
  * 如果节点是leader: 执行状态变更, 并广播到follower.
* 读请求可以由任意节点执行
  * 可以使用sync请求拿到最新的数据.

## 阶段0: Leader Election

选举出一个准leader(prospective leader), 在广播阶段准leader才成为真正的leader(established leader).

## 阶段1: Discovery

follower与准leader通信, 同步最新收到的事务提案: 让准leader发现当前大多数接受的最新事务提案.

准leader生成新的epoch, 让follower接受并更新器accpetedEpoch.

## 阶段2: Synchronization

利用发现阶段leader找到的最新的事务提案, 同步ensemble中的副本. 只有大多数节点同步完成后, 准leader才会成为真正的leader.

follower只会接受zxid比自己的lastZxid大的提案.

## 阶段3: Broadcast

ensemble可以对外提供事务服务: leader已类似2PC协议的方式广播事务.

新节点加入时, 需要做同步.

# Internal - Atomic Broadcast
* [link](https://zookeeper.apache.org/doc/current/zookeeperInternals.html)


- `myid`: service id.
- `all_server_count`: all server count.
- `zxid`: transaction id, 2 32-bit parts `(epoch, count)`, reflects total ordering.

Each time a new leader comes into power it will have its own `epoch` number.
We have a simple algorithm to assign a unique `zxid` to a proposal: the leader simply increments the `zxid` to obtain a unique zxid for each proposal.

- server state: `LOOKING`, `FOLLOWING`, `LEADING`, `OBSERVING`
- 提案proposal

```
// leader election(LE) proposal - 选主提案
<proposal>=(
epoch,
current_server_state,
self_myid,    // my knowledge
self_max_zxid,
vote_myid,    // voting
vote_max_zxid
)

// LE proposal bookkeeping - 投票箱
<current_epoch>
<bookkeeping>=
[
  (voter_myid, candidate_myid)
]
```


## Leader Activation

### FastLeaderElection

> Actions performed by the servers.

(1) initial/preferred LE proposal: voting to itself. - 给自己投票

```
<self_proposal>=(epoch, LOOKING, myid, max_zxid, myid, max_zxid)
boradcast <self_proposal> // 广播提案
```

(2) update proposal bookkeeping - 更新投票箱/修改提案

```
if <proposal>.epoch < <current_epoch>:      // 1. 旧epoch提案: 丢弃
  drop <proposal>
else if <proposal>.epoch > <current_epoch>: // 2. 新epoch提案: 
  <current_epoch> = <proposal>.epoch // 更新currentEpoch
  update and resend <self_proposal>  // 修改提案, 广播
else:                                       // 3. 当前epoch的提案
  if <proposal>.vote_max_zxid < <self_proposal>.vote_max_zxid:      // 3.1 know less 旧提案
    add/update <proposal> to <bookkeeping>
  else if <proposal>.vote_max_zxid > <self_proposal>.vote_max_zxid: // 3.2 know more 新提案
    <self_proposal>.vote_myid = <proposal>.vote_myid                // 修改提案, 广播
    <self_proposal>.vote_max_zxid = <proposal>.vote_max_zxid
    resend <self_proposal>
  else:                                                             // 3.3 know same: order by vote_myid
    if <proposal>.vote_myid < <self_proposal>.vote_myid:
      add/update <proposal> to <bookkeeping>
    else:                                                           // 比我的vote_myid大: 修改提案, 广播
      <self_proposal>.vote_myid = <proposal>.vote_myid
      resend <self_proposal>
```

(3) determine server state - 确定状态

```
SELECT <proposal>.vote_myid, COUNT(<proposal>.vote_myid) AS CANDIDATE_COUNT
FROM <bookkeeping>
GROUP BY <proposal>.vote_myid

if found any CANDIDATE_COUNT > (all_server_count / 2):  // 超过半数
  // believe the state server (<proposal>.vote_myid) = LEADING
  if myid = <proposal>.vote_myid:
    my state = LEADING
    sync with follower                    // 与follower同步
    <current_epoch> = <current_epoch> + 1 // 新的epoch
    send NEW_LEADER proposal
    keep eye on HEARTBEATs of followers // 准备处理心跳
  else:
    my state = FOLLOWING
    prepare to send HEARTBEAT to leader // 准备发送心跳
```

### Fail-over

- Follower restart

```
 this follower
<self_proposal>=(epoch, LOOKING, myid, max_zxid, myid, max_zxid)
boradcast <self_proposal>

 leader
<proposal>=(epoch, LEADING, myid, max_zxid, myid, max_zxid)

 other followers
<proposal>=(epoch, FOLLOWING, myid, max_zxid, leader.myid, leader.max_zxid)
```

```
 this follower
mystate = FOLLOWING
```

- Leader restart

```
 followers
if find leader down through HEARTBEAT:
  trigger FastLeaderElection

 leader back online
<self_proposal>=(epoch, LOOKING, myid, max_zxid, myid, max_zxid)
branches:
  - find the new leader
  - leader election is in process
```


## Active Messaging

ZooKeeper messaging operates similar to a classic two-phase commit.

All communication channels are FIFO, so everything is done in order. Specifically the following operating constraints are observed:

- (1) The leader sends proposals to all followers using the same order. Moreover, this order follows the order in which requests have been received. Because we use FIFO channels this means that followers also receive proposals in order.
- (2) Followers process messages in the order they are received. This means that messages will be `ACK`ed in order and the leader will receive `ACK`s from followers in order, due to the FIFO channels. It also means that if message `m` has been *written to non-volatile storage*, all messages that were proposed before m have been written to non-volatile storage.
- (3) The leader will issue a `COMMIT` to all followers as soon as a quorum of followers have `ACK`ed a message. Since messages are `ACK`ed in order, `COMMIT`s will be sent by the leader as received by the followers in order.
- (4) `COMMIT`s are processed in order. Followers deliver a proposal message when that proposal is committed.

# Conventions

# 应用

## 数据模型

* ZNode
* Time
* Stat

## ZNode

* 持久的节点: PERSISTENT
* 临时的节点: EPHEMERAL
* 持久的顺序目录节点
* 临时的顺序目录节点: EP

## Sessions

## Watches

## Distributed Lock

- znode: Persistent/Ephemeral, Sequence/Non-sequence
- Watch: attach a one-time watch on read operation; when the watched node is updated, a notidication is send to the watcher.

# More

ref:
- 深入浅出Zookeeper（一） Zookeeper架构及FastLeaderElection机制: http://www.jasongj.com/zookeeper/fastleaderelection/
- 深入浅出Zookeeper（二） 基于Zookeeper的分布式锁与领导选举: http://www.jasongj.com/zookeeper/distributedlock/

## UI
- [ZooKeeper Assistant](https://www.redisant.com/za): ZooKeeper Desktop GUI, 免费版本只能连接本地.
- [PrettyZoo](https://github.com/vran-dev/PrettyZoo): archived. with zkCli.sh.
- [ZooNavigator](https://github.com/elkozmon/zoonavigator): Web-based ZooKeeper UI / editor / browser. ZooKeeper versions 3.4.x and 3.5.x are currently supported.